In [143]:
import cv2
#유튜브 영상 받기 위한
import pafy
url = "https://www.youtube.com/watch?v=FksYBwUjJZc"
video = pafy.new(url)
best = video.getbest(preftype="mp4")
import numpy as np

In [144]:
#ml5 사전설정
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
import argparse #사용 안함
import posenet
import easydict
args = easydict.EasyDict({
    "model":101,
    "cam_id":best.url,
    "cam_width":1280,
    "cam_height":720,
    "scale_factor":0.712,
    "file":None
})

In [145]:
def posenet_search():
    with tf.Session() as sess: #텐서플로우의 세션을 변수에 정의
        model_cfg, model_outputs = posenet.load_model(args.model, sess)  #model_outputs는 텐서 객체들의 리스트
        output_stride = model_cfg['output_stride']

        if args.file is not None:
            cap = cv2.VideoCapture(args.file)
        else:
            cap = cv2.VideoCapture(args.cam_id)
        cap.set(3, args.cam_width)
        cap.set(4, args.cam_height)

        start = time.time()
        frame_count = 0
        
        peaple_count=1
        
        bf_keyscores=np.zeros((peaple_count,17),float)
        bf_keycoords=np.zeros((peaple_count,17,2),float)
        min_pose_score=0.15
        min_part_score=0.1
        
        while True:
            input_image, display_image, output_scale = posenet.read_cap(
                cap, scale_factor=args.scale_factor, output_stride=output_stride) #영상

            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=peaple_count,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            #---------------------------------------------------------------------------------------------------
            out_img = display_image
            
            adjacent_keypoints = []
            cv_keypoints = []
            
            
            for ii, score in enumerate(pose_scores):
                for jj,(ks, kc) in enumerate(zip(keypoint_scores[ii, :], keypoint_coords[ii, :, :])):
                    if ks > min_pose_score:
                        print(jj)
                        bf_keyscores[ii][jj]=ks
                        bf_keycoords[ii][jj]=kc
            
            for ii, score in enumerate(pose_scores):
                if score < min_pose_score:
                    continue
                results = []
                k_s= bf_keyscores[ii, :]
                k_c= bf_keycoords[ii, :, :]
                
                for left, right in posenet.CONNECTED_PART_INDICES: #선찾기
                    if k_s[left] < min_part_score or k_s[right] < min_part_score:
                        continue
                    results.append(np.array([k_c[left][::-1], k_c[right][::-1]]).astype(np.int32),)
                new_keypoints = results
                adjacent_keypoints.extend(new_keypoints)

                for ks, kc in zip(k_s, k_c):#점찾기
                    if ks < min_part_score:
                        continue
                    cv_keypoints.append(cv2.KeyPoint(kc[1], kc[0], 10. * ks))

            out_img = cv2.drawKeypoints(
                out_img, cv_keypoints, outImage=np.array([]), color=(255, 255, 0),
                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            overlay_image = cv2.polylines(out_img, adjacent_keypoints, isClosed=False, color=(255, 255, 0))


            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

        print('Average FPS: ', frame_count / (time.time() - start))

In [136]:
tester

''

In [146]:
if __name__ == "__main__":
    tester=""
    posenet_search()

5
8
11
12
13
14
4
5
6
7
8
11
12
13
14
15
16
5
7
8
9
10
11
12
13
14
15
16
5
6
8
9
10
11
12
13
14
15
16
5
6
7
8
9
10
11
12
13
14
15
16
5
6
7
8
9
10
11
12
13
14
15
16
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
5
6
7
8
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
4
5
6
7
8
9


13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
5
6
7
8
10
11
12
13
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7

14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9

9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
0
1
2
3

In [67]:
tester

array([[[ 89.41565265, 477.89332035],
        [ 85.21390247, 476.73929363],
        [ 84.1608197 , 474.11482983],
        [ 82.26847645, 475.1416855 ],
        [ 80.53561897, 463.57951141],
        [108.88270638, 488.94732571],
        [103.50121806, 441.70521284],
        [145.0067388 , 485.77000675],
        [135.40222006, 405.85336392],
        [172.15211341, 483.27125328],
        [164.75766534, 418.62908514],
        [184.81314522, 473.90211755],
        [182.32958248, 436.02677394],
        [227.12629233, 465.36507044],
        [238.63024864, 439.03344237],
        [274.23237905, 462.38760695],
        [279.77480091, 434.26282097]],

       [[ 65.61814329, 203.75643622],
        [ 59.86006881, 209.86520644],
        [ 59.76468454, 198.46651005],
        [ 65.45056905, 216.53073589],
        [ 63.99306294, 191.4357467 ],
        [ 98.83604695, 222.54561426],
        [ 96.3699556 , 179.73433259],
        [141.66668198, 234.6595285 ],
        [134.27760058, 172.85333374],
        [1

In [86]:
li=[]
li.append(1)
li.append(4)
li.append(None)
li

[1, 4, None]